In [ ]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mclr
import dill
from copy import deepcopy

In [ ]:
with open('menu_transfer.pickle', 'rb') as infile:
    menu = dill.load(infile)

In [ ]:
entry = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

In [ ]:
pairs = {0: {'O', 'Z'},
 1: {'H', 'J'},
 2: {'Q', 'W'},
 3: {'G', 'H'},
 4: {'A', 'F'},
 5: {'R', 'T'},
 6: {'A', 'C'},
 7: {'B', 'T'},
 8: {'E', 'T'},
 9: {'A', 'T'},
 10: {'J', 'U'},
 11: {'J', 'T'},
 12: {'A', 'I'},
 13: {'D', 'F'},
 14: {'Q', 'U'},
 15: {'L', 'X'},
 16: {'C', 'D'},
 17: {'A', 'O'},
 18: {'V', 'Y'}}

In [ ]:
pairs = {k:list(v) for k,v in pairs.items()}

In [ ]:
edges = [(v[0],v[1],{'label':str(k)}) for k,v in pairs.items()]
## ,{'position':str(k)}
# edges

In [ ]:
edges

In [ ]:
MG = nx.MultiGraph()

In [ ]:
# for p in edges:
#     MG.add_edge(u_for_edge=p[0], v_for_edge=p[1], label=p[2]['label'])

In [ ]:
MG.add_edges_from(edges,)

In [ ]:
# MG.edges

In [ ]:
# MG.nodes

In [ ]:
pos = nx.spring_layout(MG,k=0.4,scale=1)

In [ ]:
pos

In [ ]:
fig, ax = plt.subplots(figsize=(8,8))

nx.draw_networkx(MG,pos=pos)

labels = nx.get_edge_attributes(MG,'label')
labels = {(k[0],k[1]):v for k,v in labels.items()}  ## doesnt' seem to be able to deal with labels for multiples edges
edge_labels = nx.draw_networkx_edge_labels(MG,pos=pos,edge_labels=labels)

In [ ]:
# G = nx.Graph()

In [ ]:
# G.add_edges_from(edges)

In [ ]:
# G.edges

In [ ]:
# pos = nx.spring_layout(G) # positions for all nodes

# # nodes
# nx.draw_networkx_nodes(G,pos,node_size=400)

# # labels
# nx.draw_networkx_labels(G,pos,font_size=20,font_family='sans-serif')

# # edges
# nx.draw_networkx_edges(G,pos,edgelist=edges, width=6)

# # weights
# labels = nx.get_edge_attributes(G,'label')
# nx.draw_networkx_edge_labels(G,pos,edge_labels=labels)

In [ ]:
menu

In [ ]:
edges[:3]

In [ ]:
for c in entry:
    print(c)

In [ ]:
bombes = [k  if type(k) == int else 'REG' for k in menu.keys()]
bchars = [ch for ch in entry]
# bminimenu = {b:deepcopy(menu[b]) for b in bombes}
# bminimenu
print(bombes)
print(bchars)

In [ ]:
# cminimenu = deepcopy(bminimenu)
# for k,v in bminimenu.items():
#     for inorout in ['conx_in','conx_out']:
#         for n,l in v[inorout].items():

#             if n not in bombes:
#                 del cminimenu[k][inorout][n]
# bminimenu = deepcopy(cminimenu)
# del cminimenu
# bminimenu

In [ ]:
iomap = {'in':'I', 'out':'O', 'conx_in':'I', 'conx_out':'O'}

In [ ]:
## base graph just treating each scrambler as a single node
BG = nx.Graph()
BG.add_nodes_from(bombes)
BG.nodes

In [ ]:
base_edges = set()
for scrambler,descriptor_dict in menu.items():
    if scrambler == 'config':
        pass
    else:
        for inorout in ['conx_in','conx_out']:
            for connected_scrambler,ior in descriptor_dict[inorout].items():  
                ## the dictionary of connections that is the value for each 'conx_in/out' keys
    #             print(scrambler, connected_scrambler)
                if connected_scrambler == 'config':
                    pass
                else:
                    base_edges.add(frozenset([scrambler,connected_scrambler]))
            
base_edges = [tuple(be) for be in base_edges]  # turn set of frozensets into list of tuples
print(base_edges)

In [ ]:
BG.add_edges_from(base_edges)
BG.edges

In [ ]:
# base_pos = nx.spring_layout(BG,scale=0.05)
base_pos = nx.circular_layout(BG,scale=1)
# base_pos

In [ ]:
base_pos

In [ ]:
fig, ax = plt.subplots(figsize=(12,8))

nx.draw_networkx(BG,pos=base_pos)

In [ ]:
TG = nx.Graph()

In [ ]:
for bid in bombes:
    for i in iomap.values():
        for ch in entry:
#             this = f"{bid}-{i}-{ch}"
            TG.add_node(f"{bid}-{i}-{ch}")
# del this

In [ ]:
# TG.nodes

In [ ]:
grey = mclr.to_rgba('grey',0.5)
red = mclr.to_rgba('red',0.9)

In [ ]:
edges = set()
for scrambler,v in menu.items():
    if scrambler == 'config':
        pass
    else:
        for inorout in ['conx_in','conx_out']:
            first_node = f"{scrambler}-{iomap[inorout]}-"
            for connected_scrambler,ior in v[inorout].items():
                second_node = f"{connected_scrambler}-{iomap[ior]}-"
                for ch in bchars:
                    edges.add(frozenset([first_node+ch,second_node+ch]))

edges = [list(fs) for fs in edges]
for e in edges:
    e.append({'color': grey})
edges = [tuple(fs) for fs in edges]
# print(edges)

In [ ]:
TG.add_edges_from(edges)

In [ ]:
# TG.edges

In [ ]:
# %%time
width_of_bombe = 0.07
height_of_bombe = 0.04

wrange_of_letters = list(np.linspace(-0.5*width_of_bombe,0.5*width_of_bombe,26))

manual_pos = {}

for nd in TG.nodes():
    bid,io, ch = nd.split('-')
    try:
        bid = int(bid)
    except:
        pass
    x,y = base_pos[bid]
    
    if io == 'I':
        y += -0.5 * height_of_bombe
    else:
        y += 0.5 * height_of_bombe
        
    x += wrange_of_letters[entry.index(ch)]
    manual_pos[nd] = np.array([x,y])

# manual_pos

In [ ]:
# TG.edges[('6-I-A', '17-I-A')]['color'] = red

In [ ]:
colors = [TG[u][v]['color'] for u,v in TG.edges()]
# colors

In [ ]:
fig, ax = plt.subplots(figsize=(15,10))

nx.draw_networkx_labels(BG,pos=base_pos)

nx.draw_networkx_edges(TG,pos=manual_pos,edge_color=colors)
nx.draw_networkx_nodes(TG,pos=manual_pos, node_size=5)

plt.show()

In [ ]:
# manual_pos

In [ ]:
nx.draw_kamada_kawai(TG)